# 빅데이터 수집시스템 설계하기 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import quote
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#### [문항01] 경찰 민원포탈 > 경찰관서 찾기 > 목록에서 찾기 > 서울에서 서울 소재 31개의 경찰서의 이름과 주소를 크롤링하여 데이터프레임을 만드세요. (40점) (단, 셀레니움을 이용해서 가져오지 않고 다른 사이트를 이용해서 가져오면 감점 20점)

- 1-1. 데이터 가져오기 (10점)

In [2]:
chromedriver = '../05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [3]:
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
driver.get(url)
time.sleep(2)

- 1-2. 원하는 데이터를 찾고, 하나를 선택해서 원하는 정보 추출하기 (10점)

In [4]:
# 목록에서 찾기
driver.find_element_by_css_selector('#tab3_1').click()
time.sleep(2)

In [5]:
# 서울소재 경찰서
driver.find_element_by_css_selector('#indexedA').click()
time.sleep(2)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('#minwonTbody > tr')
len(trs)

5

In [7]:
tr = trs[0]

In [8]:
# 관서
name = tr.select_one('#minwonTbody > tr > td').get_text().strip()
name = name.split()[-1]
name

'서울중부경찰서'

In [9]:
# 주소(도로명/지번)
addr = tr.select_one('#minwonTbody > tr > td.title').get_text().strip()
addr

'서울특별시 중구 수표로 27'

- 1-3. 반복문을 사용하여 모든 페이지의 데이터 가져오기 및 오류 수정하기 (10점)

In [10]:
# 1 페이지 반복문
lines = []
for tr in trs:
    name = tr.select_one('#minwonTbody > tr > td').get_text().strip()
    name = name.split()[-1]
    addr = tr.select_one('#minwonTbody > tr > td.title').get_text().strip()
    # print(len(trs))
    lines.append([name, addr])

In [11]:
# 모든 페이지 가져오기 (7 페이지 반복문)
for i in range(1,7):
        driver.find_element_by_xpath('//*[@id="pagination"]/div/a[9]').click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        trs = soup.select('#minwonTbody > tr')
        # print(len(trs))
        for tr in trs:
                name = tr.select_one('#minwonTbody > tr > td').get_text().strip()
                name = name.split()[-1]
                addr = tr.select_one('#minwonTbody > tr > td.title').get_text().strip()
                lines.append([name, addr])

- 1-4. 데이터 프레임으로 만들고 저장하기 (10점)

In [13]:
df = pd.DataFrame(lines, columns=['관서','주소'])
df

,관서,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16
5,서울용산경찰서,서울특별시 용산구 원효로89길 24
6,서울성북경찰서,서울특별시 성북구 보문로 170
7,서울동대문경찰서,서울특별시 동대문구 약령시로 21길 29
8,서울마포경찰서,서울특별시 마포구 마포대로 183
9,서울영등포경찰서,서울특별시 영등포구 국회대로 608


In [14]:
df.to_csv('서울경찰관서.csv', index=False)

In [19]:
driver.close()

#### [문항02] 1번에서 구한 경찰서의 도로명 주소를 가지고, 카카오로컬 API를 이용하여 경찰서의 위도, 경도 정보를 구한 후 데이터프레임에 추가하세요. (30점)

In [29]:
# 파일 불러오기
df = pd.read_csv('서울경찰관서.csv')
df.head()

,관서,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


In [16]:
# 서울서부경찰서 주소 변경
df['주소'][13] = '서울특별시 은평구 서오릉로 58'
df['주소'][13]

'서울특별시 은평구 서오릉로 58'

In [17]:
# 서울강동경찰서 주소 변경
df['주소'][20] = '서울특별시 강동구 양재대로96길 35'
df['주소'][20]

'서울특별시 강동구 양재대로96길 35'

- 2-1. 카카오 로컬 API 사이트에 접속하기 (10점)

In [18]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [20]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

- 2-2. 반복문을 사용하여 모든 경찰서의 위도, 경도 좌표 구하기 (10점)

In [22]:
import requests

In [23]:
res = requests.get(url)

In [24]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

- 2-3. 데이터프레임에 위도, 경도 컬럼 추가하기 (10점)

In [25]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(3)

,관서,주소,위도,경도
0,서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549


#### [문항03] 1, 2번에서 구한 정보를 바탕으로 서울 지도(Open Street Map)위에 표시하세요. (30점)

- 3-1. 서울 지도를 그릴 것 (10점)
- 3-2. 툴팁에는 경찰서명을 표시하고, 팝업에는 경찰서 주소를 표시하되 팝업창의 너비를 늘려 가로로 표시할 것 (10점)
- 3-3. 아이콘은 기본 모양이 아닌 것으로 표시하고, 지도 제목을 ‘서울소재 경찰서 위치’로 표시할 것 (10점)

In [26]:
import folium, json

In [27]:
filename = '../04.지도시각화/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [28]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=df.관서[i],
        icon=folium.Icon(color='orange', icon='star')
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>' 
map.get_root().html.add_child(folium.Element(title_html))
map